# Desigualdad y crecimiento económico en Argentina

Este proyecto explora la evolución económica y la desigualdad en Argentina. Utiliza datos de fuentes confiables para visualizar tendencias importantes y proporciona insights sobre la correlación entre el PBI per cápita y el índice de Gini a lo largo de diferentes períodos presidenciales.

Basado en el [excelente análisis](https://thevizcorner.wordpress.com/2015/11/16/desigualdad-y-crecimiento-economico-en-argentina/) de Fernando Cucchietti ([@thefercook](https://twitter.com/thefercook)).

### Fuentes
Banco Mundial, INDEC. Cada punto representa cualitativamente el 31 de Diciembre de cada año.
El BM nota que el PBI de 2012 a 2015 usa un factor de cambio combinado oficial y paralelo. 
El valor Gini de 1988, 1989, 1990 y 2015 fue interpolado entre sus valores más cercanos.

## Configuración inicial y obtención de datos

In [502]:
import plotly.graph_objects as go
import numpy as np
import pandas as pd

from util.functions import *

#### Año de corte

In [503]:
# Cualquier dato tendrá un año igual o menor al siguiente
cut_year = 2023

#### Coeficiente Gini
Datos del [Banco Mundial](https://datos.bancomundial.org/indicator/SI.POV.GINI?locations=AR)
Último dato provisorio cuarto trimestr 2023 de [Indec](https://www.indec.gob.ar/uploads/informesdeprensa/ingresos_4trim244EC0BA3D2C.pdf)


In [504]:
# gini = pd.read_csv("assets/gini.csv") 
# gini = pd.read_csv("assets/API_ARG_DS2_en_csv_v2_2164835.csv") 

# Datos del Banco Mundial (https://datos.bancomundial.org/indicator/SI.POV.GINI?locations=AR)
gini = pd.read_csv("assets/GINI_Arg_Banco_Mundial.csv") 
gini.head()

,country,1960,1961,1962,1963,1964,1965,1966,1967,1968,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Argentina,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,41.8,41.8,42.3,41.4,41.7,43.3,42.7,42.4,40.7,43.4


In [505]:
# Me aseguro de que sólo quede Argentina en los datos
gini_arg = gini.loc[(gini.country == "Argentina")].copy()
# gini_arg = gini.loc[(gini["Country Name"] == "Argentina")].copy()

gini_arg.drop(columns='country', inplace=True)
gini_arg.tail(5)

,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,41.8,41.8,42.3,41.4,41.7,43.3,42.7,42.4,40.7,43.4


#### PBI per Cápita
En dólares al 2015. Datos del [Banco Mundial](https://data.worldbank.org/indicator/NY.GDP.PCAP.KD?locations=AR&page=1)

In [506]:
# Datos del Banco Mundial (https://data.worldbank.org/indicator/NY.GDP.PCAP.KD?locations=AR&page=1)
gdp = pd.read_csv("assets/PIB_per_capita_arg_2015_constant_banco_mundial_20240628.csv")
# gdp = pd.read_csv("assets/PIB_per_capita_arg__2015_constant_banco_mundial.csv")
# gdp = pd.read_csv("assets/income_per_person_gdppercapita_ppp_inflation_adjusted.csv")
# gdp = pd.read_csv("assets/PIB_per_capita_PPP_2011_Banco_Mundial.csv")
# gdp = pd.read_csv("assets/gdppercapita_us_inflation_adjusted.csv")

# Me aseguro de que sólo quede Argentina en los datos
gdp_arg = gdp.loc[(gdp.country == "Argentina")].copy()

gdp_arg.drop(columns='country', inplace=True)

gdp_arg.head()

,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,7410.305029,7687.517338,7498.840061,6986.564802,7572.231166,8241.529337,8059.450431,8187.373351,8450.037064,9126.186479,...,13567.948416,13789.060425,13360.211835,13595.037355,13105.397163,12716.224204,11346.652113,12444.318267,12940.738435,12625.469551


#### Preparo dataset para cubrir fechas mayores a 1985 y menores o iguales al año de corte

In [507]:
# Preparo dataset para cubrir fechas mayores a 1985 y menores o iguales al año de corte
gdp_arg2 = pd.melt(gdp_arg, var_name="year", value_name="gdp")
gdp_arg2.year = pd.to_numeric(gdp_arg2.year)
gdp_arg2 = gdp_arg2.loc[(gdp_arg2.year > 1985) & (gdp_arg2.year <= cut_year)]
gdp_arg2.reset_index(drop=True, inplace=True)
gdp_arg2.head()

,year,gdp
0,1986,9397.883608
1,1987,9507.339813
2,1988,9262.963927
3,1989,8473.026569
4,1990,8144.494294


In [508]:
gini_arg2 = pd.melt(gini_arg, var_name="year", value_name="gini")
gini_arg2.year = pd.to_numeric(gini_arg2.year)
gini_arg2 = gini_arg2.loc[(gini_arg2.year > 1985) & (gini_arg2.year <= cut_year)]
gini_arg2.drop(columns='year', inplace=True)
gini_arg2.reset_index(drop=True, inplace=True)
gini_arg2.head()

,gini
0,42.8
1,45.3
2,45.3
3,46.1
4,46.1


#### Junto ambos datasets (Gini y PBI per cápita)

In [509]:
arg = pd.concat([gdp_arg2, gini_arg2], axis=1)
arg.tail()

,year,gdp,gini
33,2019,12716.224204,43.3
34,2020,11346.652113,42.7
35,2021,12444.318267,42.4
36,2022,12940.738435,40.7
37,2023,12625.469551,43.4


In [510]:
arg.describe()

,year,gdp,gini
count,38.000000,38.000000,38.000000
mean,2004.500000,11401.216967,45.634211
std,11.113055,1825.405503,3.570697
min,1986.000000,8144.494294,40.700000
25%,1995.250000,9960.275286,42.700000
50%,2004.500000,11243.981392,45.300000
75%,2013.750000,13064.232481,48.325000
max,2023.000000,14200.269889,53.800000


In [511]:
arg = arg.reset_index(drop=True)
# arg

#### Cargo los períodos presidenciales

In [512]:
presidentials2 = pd.read_csv("assets/presidentials.csv")
presidentials2 = presidentials2.loc[presidentials2.country == "Argentina"].copy()
presidentials2.drop(columns='country', inplace=True)
presidentials2.head(11)

,president,from,to,annotation_x,annotation_y,note
0,Alfonsín,1986-01-01,1989-07-08,9100,43.50,NaN
1,Menem (1),1989-07-08,1995-07-08,9600,46.10,NaN
2,Menem (2),1995-07-08,1999-12-10,11610,49.00,NaN
3,De la Rúa,1999-12-10,2001-12-21,10650,52.50,NaN
4,5 Presidentes,2001-12-21,2002-01-02,10100,53.70,NaN
5,Duhalde,2002-01-02,2003-05-25,8600,53.00,NaN
6,Kirchner,2003-05-25,2007-12-10,11800,47.20,NaN
7,Fernández C. (1),2007-12-10,2011-12-10,14300,43.40,NaN
8,Fernández C. (2),2011-12-10,2015-12-10,14700,41.50,NaN
9,Macri,2015-12-10,2019-12-10,13400,42.65,NaN


Tomo las fechas de cada período presidencial y las traduzco en la posición que les corresponde en el gráfico de Gini/PBI per cápita

In [513]:
import math
df = arg.copy()

df["president"] = presidentials2["president"].head(1).values[0]

for presi in presidentials2["president"].unique():
    presi_row = presidentials2[presidentials2.president == presi]
    if not pd.isnull(presi_row["to"].values[0]):
        str_to = presi_row["to"].values[0]
        floor_to = get_year_from_datestr(str_to) - 1
        percent_to = get_date_year_percentage(str_to)

        row_num = df.index[df["year"] == floor_to][0] + 1
        row_val = df.loc[df["year"] == floor_to].copy()
        row_val_next_year = df.loc[df["year"] == (floor_to+1)].copy()
        row_val.year = floor_to + percent_to
        row_val.gdp = row_val.gdp.values[0] + (row_val_next_year.gdp.values[0] - row_val.gdp.values[0]) * percent_to
        row_val.gini = row_val.gini.values[0] + (row_val_next_year.gini.values[0] - row_val.gini.values[0]) * percent_to

        df = insert_row(row_num, df, row_val.values[0]) 

    str_from = presi_row["from"].values[0]
    floor_from = get_year_from_datestr(str_from) - 1
    percent_from = get_date_year_percentage(str_from)
    df.loc[df.year > (floor_from + percent_from), "president"] = presi

#### Por último cargo el dataset de eventos

In [514]:
events = pd.read_csv("assets/events.csv")
events.head()

,date,scope,country,event_name,description,xshift,yshift,ax,ay
0,1987-01-15,country,Argentina,Plan Austral,NaN,10,0,60,0
1,1989-04-15,country,Argentina,,NaN,0,-10,-30,10
2,1990-03-01,country,Argentina,Hiperinflación,NaN,0,-10,0,40
3,1991-03-27,country,Argentina,Convertibilidad,NaN,0,15,-10,-20
4,1994-12-10,region,Argentina,Efecto Tequila,NaN,10,0,60,30


## Gráfico

#### Código

In [515]:
all = df

hoverinfo = ['text']
line_shape = 'spline'
mode = 'markers+text'
mode_presis = 'lines'
marker=dict(
    color='white',
    size=6,
    line=dict(
        color='black',
        width=2
    )
)

# Acomodo marcadores de años
years = arg.copy()
years["textposition"] = "top left"
years.loc[years.year.isin([1991, 1993, 2015, 2016]), "textposition"] = "top center"
years.loc[years.year.isin([1988, 1999]), "textposition"] = "bottom center"
years.loc[years.year.isin([1989, 1992, 1994, 1998, 2000, 2001, 2003, 2004, 2005, 2006, 2007, 2008, 2010, 2011, 2018]), "textposition"] = "top right"
years.loc[years.year.isin([1986, 1990, 2014, 2021, 2022]), "textposition"] = "bottom left"
years.loc[years.year.isin([1987, 2012, 2019]), "textposition"] = "middle right"
years.loc[years.year.isin([1997, 2013, 2017]), "textposition"] = "bottom right"
years.loc[years.year.isin([2023]), "textposition"] = "middle left"
textpositions = years["textposition"].values
years = years["year"].values


showlegend=False

fig = go.Figure()

import plotly.express as px
# Uso una paleta Vívida para los colores
colors = px.colors.qualitative.Vivid.copy()

presi_tail = pd.DataFrame()
for presi_key in presidentials2['president'].unique():
    presi_row = presidentials2[presidentials2.president == presi_key]
    presi_df = all.loc[(all['president'] == presi_key)].copy()

    # Agrego el último valor de una presidencia en la otra para segmentar correctamente por color
    if not presi_tail.empty:
        presi_df = pd.concat([presi_tail, presi_df])
        # presi_df = presi_tail.append(presi_df)

    presi_tail = presi_df.tail(1)

    presi_color = colors.pop()

    fig.add_trace(go.Scatter(
        x = presi_df.gdp,
        y = presi_df.gini,
        hoverinfo = hoverinfo,
        line_shape = line_shape,
        line=dict(color = presi_color, width = 5),
        mode = mode_presis,
        showlegend = showlegend
    ))

    

    # Texto con los nomnbres de los presidentes
    fig.add_annotation(
        x=presi_row["annotation_x"].values[0], 
        y=presi_row["annotation_y"].values[0],
        text = ("<b>%s</b>" % presi_key),
        font=dict(
            family="Helvetica, Verdana",
            size=12,
            color=presi_color
            ),
        showarrow=False,
        xref="x",
        yref="y",
        yshift=10,
    )

# Con puntos
fig.add_trace(go.Scatter(
    x = arg.gdp,
    y = arg.gini,
    text = years,
    texttemplate = "<b>%{text}</b>",
    hoverinfo = hoverinfo,
    hovertemplate = "Finales de año: <b>%{text}</b><br>PBI PPP: <b>$%{x:.0f}</b><br>Gini: <b>%{y:.2f}</b><br>",
    line_shape = line_shape,
    mode = mode,
    marker = marker,
    textposition = textpositions,
    showlegend = showlegend
))

fig.update_traces(textfont = {'family': "Helvetica, Verdana", 'size': 11, 'color': "#090933"})

fig.update_layout(
    height=800,
    width=1000,
    title_text= '<b>Evolución económica y desigualdad</b> (Argentina %s - %s)' % (int(all.year.min()), int(all.year.max())),
    font=dict(
        family="Helvetica, Verdana",
        # size=18,
    ),
    paper_bgcolor='#ffffff',
    plot_bgcolor='#ffffff',
    hoverdistance=100, # Distance to show hover label of data point
    spikedistance=1000, # Distance to show spike
    xaxis=dict(
        # title= "<b>Producto Bruto Interno per Capita",
        linecolor='rgba(0,0,0,0)', # "#BCCCDC",  # Sets color of X-axis line
        showgrid=False,  # Removes X-axis grid lines,
        showspikes=True, # Show spike line for X-axis
        # Format spike
        spikethickness=1,
        spikedash="dot",
        spikecolor="#999999",
        spikemode="across",
        mirror=True,          # Refleja las líneas en el eje opuesto
    ),
    yaxis=dict(
        # title= "<b>Índice de desigualdad GINI</b>",
        linecolor='rgba(0,0,0,0)', # "#BCCCDC",  # Sets color of Y-axis line
        showgrid=False,  # Removes Y-axis grid lines 
        showspikes=True, # Show spike line for X-axis
        # Format spike
        spikethickness=1,
        spikedash="dot",
        spikecolor="#999999",
        spikemode="across",  
        mirror=True,          # Refleja las líneas en el eje opuesto
    ),
    legend=dict(
        # Adjust click behavior
        itemclick="toggleothers",
        itemdoubleclick="toggle",
    )
)

# Ticks en ambos lados
fig.update_layout(
    xaxis=dict(
        ticks='outside',  # Muestra las marcas hacia fuera del gráfico
        ticklen=6,        # Longitud de las marcas
        tickwidth=2,      # Ancho de las marcas
        tickcolor='black', # Color de las marcas
        mirror='ticks',   # Refleja las marcas en el eje opuesto
        # ... otras configuraciones para xaxis ...
    ),
    yaxis=dict(
        ticks='outside',  # Muestra las marcas hacia fuera del gráfico
        ticklen=6,        # Longitud de las marcas
        tickwidth=2,      # Ancho de las marcas
        tickcolor='black', # Color de las marcas
        mirror='ticks',   # Refleja las marcas en el eje opuesto
        # ... otras configuraciones para yaxis ...
    ),
    # ... otras configuraciones para fig.update_layout ...
)

# Eje vertical (Título)
# #####################
fig.add_annotation(
    xref="paper",
    yref="paper",
    x=0.9,
    y=0.935,
    align="right",
    text="<b>Índice de desigualdad GINI</b><br>Valores más altos indican mayor desigualdad",
    showarrow=False,
    font=dict(
        family="Helvetica, Verdana",
        size=13
    )
)

# Flecha vertical
fig.add_annotation(
    x=0.91,
    y=0.96,
    xref="paper",
    yref="paper",
    showarrow=True,
    arrowside="end+start",
    align="center",
    arrowhead=2,
    arrowsize=1,
    arrowwidth=2,
    arrowcolor="black",
    ax=0,
    ay=74,
    opacity=1
)

# Eje horizontal (Título)
# #######################
fig.add_annotation(
    xref="paper",
    yref="paper",
    x=0.075,
    y=0.016,
    align="center",
    text="<b>Producto Bruto Interno per Capita</b><br>USD ajustados por inflación a valor constante 2015",
    showarrow=False,
    font=dict(
        family="Helvetica, Verdana",
        size=13
    )
)

# Flecha horizontal
fig.add_annotation(
    x=0.15,
    y=0.09,
    xref="paper",
    yref="paper",
    showarrow=True,
    arrowside="end+start",
    align="center",
    arrowhead=2,
    arrowsize=1,
    arrowwidth=2,
    arrowcolor="black",
    ax=180,
    ay=0,
    opacity=1
)


# Eventos del país y mundiales

for idx, event in events.iterrows():
    # print(event)
    str_to = event.date
    floor_to = get_year_from_datestr(str_to) - 1
    percent_to = get_date_year_percentage(str_to)

    row_val = df.loc[df["year"] == floor_to].copy()
    row_val_next_year = df.loc[df["year"] == (floor_to+1)].copy()

    fig.add_annotation(
        x = row_val.gdp.values[0] + (row_val_next_year.gdp.values[0] - row_val.gdp.values[0]) * percent_to ,
        y = row_val.gini.values[0] + (row_val_next_year.gini.values[0] - row_val.gini.values[0]) * percent_to,
        text = ('<i>%s</i>' %event.event_name),
        font=dict(
            family="Helvetica, Verdana",
            size=12,
            color= "grey"
            ),
        showarrow=True,
        # arrowside="end",
        align="center",
        arrowhead=2,
        arrowsize=1,
        arrowwidth=1,
        arrowcolor="grey",
        ax = event.ax,
        ay = event.ay,
        xref="x", 
        yref="y",
        xshift=event.xshift,
        yshift=event.yshift
        )

# Fuentes
fig.add_annotation(
    xref="paper",
    yref="paper",
    x=-0.03,
    y=-0.11,
    align="left",
    text="<b>Fuentes:</b> Banco Mundial, INDEC. Cada punto representa cualitativamente el 31 de Diciembre de cada año.<br>El BM nota que el PBI de 2012 a 2015 usa un factor de cambio combinado oficial y paralelo. El valor Gini de 1988, 1989, 1990 y 2015 fue interpolado entre sus valores más cercanos.",
    showarrow=False,
    font=dict(
        family="Helvetica, Verdana",
        size=11,
        color= "grey"
    )
)


#### Muestro el gráfico

In [516]:
fig.show(config={"displayModeBar": False, "showTips": False}) # Remove floating menu and unnecesary dialog box

#### Guardo última ejecución del gráfico 

In [517]:
# Guardo una imágen de la última ejecución
import plotly.io as pio

fig.write_image("images/lastrun.png")